### Установка и импорт всех необходимых зависимостей

!pip install -q razdel
!pip install -q pymorphy2
!pip install -q git+https://github.com/ahmados/rusynonyms.git
!pip install -q natasha
!pip install -q pyaml-env
!pip install -q captum

In [ ]:
import os
import sys

path_to_alti = '/kaggle/input/transformer-contributions1/transformer-contributions'
if not path_to_alti in sys.path:
    sys.path.append(path_to_alti)

from src.utils_contributions import *
from src.contributions import ModelWrapper, ClassificationModelWrapperCaptum, interpret_sentence, occlusion

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

import nltk
from nltk.corpus import stopwords
import re
import pymorphy2
from razdel import tokenize
from razdel import sentenize
import string
from natasha import (
    MorphVocab,
    NewsMorphTagger,
    NewsEmbedding,
    Segmenter,
    NewsSyntaxParser,
    Doc
)

import torch
import tensorflow_hub as hub
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import numpy as np
from sklearn.metrics import f1_score

from tqdm import tqdm
from typing import *

from lime.lime_text import LimeTextExplainer
import shap

nltk.download('stopwords')
nltk.download('punkt')
rus_stopwords = stopwords.words('russian')
punctuation = list(string.punctuation)

### Работа с данными (kaggle)

In [ ]:
datasets_folder = '/kaggle/input/sw-datasets/Russian-Sentiment-Analysis-Evaluation-Datasets'
datasets = ['SentiRuEval-2015-telecoms', 'SentiRuEval-2015-banks', 'SentiRuEval-2016-banks', 'SentiRuEval-2016-telecoms']
samples = ['test.xml', 'train.xml', 'test_etalon.xml']

In [ ]:
def extract_data(path: str) -> pd.DataFrame:
    """
    функция для извлечения данных из xml
    """
    tree = ET.parse(path)
    root = tree.getroot()
    DataFrame = dict()
    database = root.findall('database')[0]
    DataFrame_columns = list()

    for idx, table in enumerate(database.findall('table')):
        for column in table.findall('column'):
            DataFrame[column.attrib['name']] = list()
            DataFrame_columns.append(column.attrib['name'])
        if idx == 0:
            break

    for table in database.findall('table'):
        for column in table.findall('column'):
            DataFrame[column.attrib['name']].append(column.text)

    data = pd.DataFrame(DataFrame, columns=DataFrame_columns)
    return data

# инициализация всех путей (kaggle)
banks_dataset = datasets[2]
path2samples = os.path.join(datasets_folder, banks_dataset)
banks = ['sberbank', 'vtb', 'gazprom', 'alfabank', 'bankmoskvy', 'raiffeisen', 'uralsib', 'rshb']

path2test = os.path.join(path2samples, samples[2])
data_test = extract_data(path2test)

path2train = os.path.join(path2samples, samples[1])
data_train = extract_data(path2train)

In [ ]:
def extract_text_features(data: pd.DataFrame) -> pd.DataFrame:
    """
    функция для первичной обработки текста от лишних символов
    """
    extracted_data = dict()
    extracted_data['text'] = list()
    extracted_data['0class'] = list()
    extracted_data['1class'] = list()

    for idx in range(len(data)):
        row = data.iloc[idx, :]
        banks_review = row[banks]
        unique_labels = set(banks_review)
        unique_labels.remove('NULL')

        # убираем все ненужные знаки
        filtered_text = re.sub('http[A-z|:|.|/|0-9]*', '', row['text'])
        filtered_text = re.sub('@\S*', '', filtered_text)
        filtered_text = re.sub('#|:|»|«|-|xD|;D|\"|_|/', '', filtered_text)
        filtered_text = re.sub(r'\.(?=\s)|,|(?<!\s)\.(?!\s)', ' ', filtered_text)
        filtered_text = re.sub(r'[A-Z]|[a-z]', '', filtered_text)
        filtered_text = re.sub(r'\d+', 'число', filtered_text)
        filtered_text = re.sub(r'\s+', ' ', filtered_text).strip()
        new_text = filtered_text

        # сохраняем только уникальные токены (без придатка xml NULL)
        unique_labels = list(unique_labels)
        while len(unique_labels) < 2:
            unique_labels.append(unique_labels[-1])
        extracted_data['text'].append(new_text)
        for idx, label in enumerate(unique_labels):
            text_label = int(label) + 1
            extracted_data[f'{idx}' + 'class'].append(text_label)

    extracted_data = pd.DataFrame(extracted_data)
    
    # возвращаем dataframe
    return extracted_data

extracted_val = extract_text_features(data_test)
extracted_train = extract_text_features(data_train)

In [ ]:
# пример твита из датасета
extracted_val.iloc[3308].text

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# анализ распределения таргетов на твитах
fig, axes = plt.subplots(1, 2, figsize=(8, 5))
plt.subplots_adjust(hspace=0.3, wspace=0.5)
fontsize=15

sns.countplot(data=extracted_train, x='0class', ax=axes[0])
axes[0].set_xlabel('class 0', fontsize=fontsize)
axes[0].set_ylabel('count', fontsize=fontsize)
axes[0].set_xticks([0, 1, 2], ['Neg', 'Neu', 'Pos'], fontsize=fontsize)
axes[0].grid(True)

sns.countplot(data=extracted_train, x='1class', ax=axes[1])
axes[1].set_xlabel('class 1', fontsize=fontsize)
axes[1].set_ylabel('count', fontsize=fontsize)
axes[1].set_xticks([0, 1, 2], ['Neg', 'Neu', 'Pos'], fontsize=fontsize)
axes[1].grid(True)

fig.suptitle('target distribution', fontsize=fontsize)

None

### Инициализируем модель (fine-tune) для решения нашей задачи классификации

In [ ]:
fn_model_name = "DeepPavlov/distilrubert-base-cased-conversational"

class BERTmy(torch.nn.Module):
    def __init__(
        self, model_name: str, n_classes: int, 
        use_tok_type_ids: bool, p: float=0.05
    ) -> None:
        super(BERTmy, self).__init__()
        self.rubert = transformers.AutoModel.from_pretrained(
            model_name
        )
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(
            model_name, 
            do_lower_case=True,
            add_additional_tokens=True
        )
        self.use_tok_type_ids = use_tok_type_ids
        
        hidden_size_output = self.rubert.config.hidden_size
        self.pre_classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size_output, hidden_size_output, bias=True),
            torch.nn.Dropout(p),
            torch.nn.ReLU()
        )
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size_output, hidden_size_output, bias=True),
            torch.nn.Dropout(p),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size_output, n_classes),
        )

    def forward(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor, 
        token_type_ids: torch.Tensor=None, output_attentions: bool=False,
        output_hidden_states: bool=False, return_dict: bool=True
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        
        input_dict = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'return_dict': True,
            'output_attentions': True,
            'output_hidden_states': True
        }
        if self.use_tok_type_ids and not token_type_ids is None:
            input_dict['token_type_ids'] = token_type_ids
        
        rubert_output = self.rubert(**input_dict)

        pooled = rubert_output['last_hidden_state']
        attentions = rubert_output['attentions']
        hid_states = rubert_output['hidden_states']

        output_pre_cls = self.pre_classifier(pooled[:, 0, :])
        logits = self.classifier(output_pre_cls)

        return {
            'logits': logits,
            'attentions': attentions,
            'hidden_states': hid_states
        }

In [ ]:
def load_model_hf_cls(
    model_load: str, model_type: str, 
    load_model_weights: bool=False
) -> torch.nn.Module:

    assert model_type in ['distilbert', 'bert']

    tokenizer = AutoTokenizer.from_pretrained(
        model_load, do_lower_case=True,
        add_additional_tokens=True
    )

    if load_model_weights:
        model = AutoModel.from_pretrained(model_load)
        model_config = model.config
    else:
        model_config = AutoConfig.from_pretrained(model_load)

    model_cls = AutoModelForSequenceClassification.from_config(model_config)
    
    if load_model_weights:
        if model_type == 'distilbert':
            model_cls.distilbert = model
        elif model_type == 'bert':
            model_cls.bert = model
        del model

    return model_cls, tokenizer

In [ ]:
distilbert_name = "DeepPavlov/distilrubert-base-cased-conversational"
bert_base_name = "DeepPavlov/rubert-base-cased"

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_cls = len(pd.unique(extracted_train['0class']))
load_tf = True

if load_tf:
    model_cls, tokenizer = load_model_hf_cls(
        distilbert_name, model_type='distilbert', 
        load_model_weights=True
    )
    seq_max_len = model_cls.config.max_position_embeddings
    hid_dim = model_cls.config.dim
    model_cls.dropout = torch.nn.Identity()
    model_cls.pre_classifier = torch.nn.Sequential(
        torch.nn.Linear(hid_dim, hid_dim, bias=False),
        torch.nn.Dropout(0.15),
        torch.nn.ReLU()
    )
    model_cls.classifier = torch.nn.Sequential(
        torch.nn.Linear(hid_dim, hid_dim, bias=False),
        torch.nn.Dropout(0.15),
        torch.nn.ReLU(),
        torch.nn.Linear(hid_dim, num_cls, bias=False),
    )
else:
    model_cls = BERTmy(model_name=distilbert_name, n_classes=num_cls, use_tok_type_ids=False)
    tokenizer = model_cls.tokenizer
    seq_max_len = model_cls.rubert.config.max_position_embeddings

In [ ]:
train_batch_size = 24
val_batch_size = 24

class SentimentDataTransformer(Dataset):
    # инициализация датасета
    def __init__(
        self, texts: List[str], 
        labels: List[Tuple[int, ...]]=None
    ) -> None:
        
        self.texts = texts
        self.labels = labels

    # для получения размера датасета
    def __len__(self) -> int:
        return len(self.texts)

    # для получения элемента по индексу
    def __getitem__(
        self, index: int
    ) -> Tuple[Union[str, int]]:

        if self.labels is None:
            return self.texts[index]

        text = self.texts[index]
        labels = self.labels[index]
        
        target1, target2 = labels

        return text, target1, target2

In [ ]:
class collate_fn_transformers():
    
    def __init__(
        self, tokenizer: AutoTokenizer, 
        use_labels:bool, use_tok_type_ids: bool
    ) -> None:
        
        self.tokenizer = tokenizer
        self.use_tok_type_ids = use_tok_type_ids
        self.use_labels = use_labels
        
    def __call__(self, batch):
        
        if not self.use_labels:

            texts = batch

            return self.tokenizer(
                texts, #truncation=True,
                padding=True, add_special_tokens=True,
                return_token_type_ids=self.use_tok_type_ids,
                return_tensors='pt'
            )
        
        texts, target1, target2 = zip(*batch)
        
        input_ids = self.tokenizer(
            texts, #truncation=True,
            padding=True, add_special_tokens=True,
            return_token_type_ids=self.use_tok_type_ids,
            return_tensors='pt'
        )
        target1 = torch.tensor(target1)
        target2 = torch.tensor(target2)
        
        return input_ids, target1, target2

### Инициализируем наши DataLoaders

In [ ]:
train = SentimentDataTransformer(
    texts=extracted_train['text'].tolist(),
    labels=list(zip(extracted_train['0class'], extracted_train['1class']))
)

val = SentimentDataTransformer(
    texts=extracted_val['text'].tolist(),
    labels=list(zip(extracted_val['0class'], extracted_val['1class']))
)

train_loader = DataLoader(
    train, batch_size=train_batch_size, shuffle=True,
    collate_fn=collate_fn_transformers(tokenizer=tokenizer, use_tok_type_ids=False, use_labels=True)
)
val_loader = DataLoader(
    val, batch_size=val_batch_size, shuffle=False,
    collate_fn=collate_fn_transformers(tokenizer=tokenizer, use_tok_type_ids=False, use_labels=True)
)
loaders = {
    'train': train_loader,
    'val': val_loader
}

### Дообучение модели

In [ ]:
def train_model(
    epochs: int, model: torch.nn.Module, loaders: Dict[str, DataLoader], 
    optimizer: torch.optim, scheduler: torch.optim.lr_scheduler, 
    weights_vector: torch.tensor=None, device: str='cpu'
) -> None:
    # cross entropy loss
    model = model.to(device)
    if weights_vector is None:
        weights_vector = torch.ones(size=(num_cls,), device=device)
    loss_function1 = torch.nn.CrossEntropyLoss(reduction='mean', weight=weights_vector)
    loss_function2 = torch.nn.CrossEntropyLoss(reduction='mean', weight=weights_vector)
    
    # извлечение DataLoaders
    if len(loaders) > 1:
        train_loader = loaders['train']
        val_loader = loaders['val']
        steps_per_epoch = [('train', train_loader), ('val', val_loader)]
    else:
        train_loader = loaders['train']
        steps_per_epoch = [('train', train_loader)]

    # обучение по эпохам
    for epoch in range(epochs):
        for mode, loader in steps_per_epoch:
            # сохранение статистик
            train_loss = 0
            n_correct = 0
            processed_data = 0
            
            # train/val 
            if mode == 'train':
                model.train()
                requires_grad_mode = True
            else:
                model.eval()
                requires_grad_mode = False
            
            # проход по батчам
            for inputs, trg1, trg2 in tqdm(loader):
                # обнуляем градиенты
                optimizer.zero_grad()

                # извлечение входных данных для модели
                for key, value in inputs.items():
                    inputs[key] = value.to(device)
                trg1, trg2 = trg1.to(device), trg2.to(device)
                inputs['return_dict'] = True
                
                # устанавливаем необходимость вычислять/не_вычислять градиенты
                with torch.set_grad_enabled(requires_grad_mode):
                    outputs = model(**inputs)
                    preds = torch.argmax(outputs['logits'], dim=1)

                    # настраиваем модели на конкретный target
                    if all(trg1 == trg2):
                        loss1 = loss_function1(outputs['logits'], trg1)
                        train_loss += loss1.item()
                        n_correct += torch.sum(preds == trg1).cpu().detach().numpy()
                        if mode == 'train':
                            # вычисляем градиенты и обновляем веса
                            loss1.backward()
                            optimizer.step()
                    # если у твита более чем 1 метка, то настраиваем на обе
                    else:
                        loss1 = loss_function1(outputs['logits'], trg1) * 0.5
                        loss2 = loss_function2(outputs['logits'], trg2) * 0.5
                        loss_all = loss1 + loss2
                        train_loss += loss_all.item()

                        mask_singular = trg1 == trg2
                        mask_multiple = trg1 != trg2
                        singular = preds[mask_singular]
                        n_correct += torch.sum(
                            singular == trg1[mask_singular]
                        ).cpu().detach().numpy()
                        multiple = preds[mask_multiple]
                        n_correct += torch.sum(
                            (multiple == trg1[mask_multiple]) | (multiple == trg2[mask_multiple])
                        ).cpu().detach().numpy()
                        if mode == 'train':
                            # вычисляем градиенты и обновляем веса
                            loss_all.backward()
                            optimizer.step()

                    processed_data += len(preds)

            # вычисляем ошибку и точность прогноза на эпохе
            loader_loss = train_loss / processed_data
            loader_acc = n_correct / processed_data
            print(f'{epoch + 1} epoch with {mode} mode has: {loader_loss} loss, {loader_acc} acc')
        
        # делаем шаг для sheduler оптимайзера
        scheduler.step()

In [ ]:
epochs = 3
optimizer = torch.optim.Adam([
    {'params': model_cls.pre_classifier.parameters(), 'lr': 8e-4},
    {'params': model_cls.classifier.parameters(), 'lr': 8e-4}
], lr=2e-6)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
train_model(epochs, model_cls, loaders, optimizer, scheduler, device)

In [ ]:
model_name = 'distilbert_cls.pth'

mode_process = input('Load weights? (y/n)')
if mode_process == 'n':
    torch.save(model_cls.state_dict(), model_name)
elif mode_process == 'y':
    model_cls.load_state_dict(torch.load('/kaggle/input/distilbert-weights/distilbert_cls.pth'))
else:
    assert mode_process in ['n', 'y']
model_cls.eval()
None